In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch import FloatTensor
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [ ]:
x_data = []
y_data = []

for i in range(len(x)):
    if y[i] == 0 or y[i] == 1:
        x_list = x[i].tolist()
        x_data.append(x_list)
        y_data.append(y[i])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=10,
                                                    random_state=42)

train_idx = []
test_idx = []

for i in range(len(y_train)):
    if y_train[i] == 2:
        train_idx.append(i)

for i in range(len(y_test)):
    if y_test[i] == 2:
        test_idx.append(i)

x_train = np.delete(x_train, train_idx, axis=0)
x_test = np.delete(x_test, test_idx, axis=0)

y_train = np.delete(y_train, train_idx)
y_test = np.delete(y_test, test_idx)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

Y_train = torch.as_tensor(y_train)
Y_test = torch.as_tensor(y_test)

train_set = TensorDataset(FloatTensor(x_train), Y_train)
test_set = TensorDataset(FloatTensor(x_test), Y_test)

train_set = DataLoader(train_set, batch_size=20)
test_set = DataLoader(test_set, batch_size=1)

In [ ]:
class Circuit(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4,4)
        self.fc3 = nn.Linear(4,2)
        
    def forward(self, x):
        x1 = F.relu(self.fc1(x))
        x2 = F.tanh(self.fc2(x1))+x1
        x3 = (self.fc3(x2))**2


        return x3
model = Circuit()

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:
n_epochs = 100
LR = 0.01

for epoch in range(n_epochs):
    train_loss = 0.0
    for data in train_set:

        input = data[0]
        # input = torch.tensor(input)
        # input = input.type(torch.FloatTensor)
        target = data[1]
        # target = torch.tensor(target)
        # target = target.type(torch.FloatTensor)
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss/len(x_train)
    if epoch%2 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

In [ ]:
def test(network, data):
    # tests the accuracy of the network on the dataset data and returns it (the accuracy)
    correct_count = 0
    all_count = 0

    with torch.no_grad():
        for x, y in data:
            outputs = network(x)
            _, pred = torch.max(outputs.data, 1)
            all_count += y.size(0)
            correct_count += (pred == y).sum().item()

    return correct_count / all_count

In [ ]:
print(test(model,train_set))
print(test(model,test_set))